#### Ensemble Model Execution
In this notebook, the predicted probabilities of the text classifiers will be combined with those of the image classifier (IRNV2). Hereafter, these are used to train the meta-learner (SVM), and validate on the meta-learner. This will result in a top-1 accuracy score per ensemble model

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import GridSearchCV





The predicted probabilities for both 'training_dataset(2)' and 'validation_dataset(2)' are now imported

In [3]:
## Test with probabilities of ImageModel
image_train_input = pd.read_excel("image_input_train2_ensemble.xlsx")
text_train_input = pd.read_excel("LR_Predictions_train2.xlsx")

image_val_input = pd.read_excel('image_input_val2_ensemble.xlsx')
text_val_input = pd.read_excel("LR_Predictions_val2.xlsx")

In [5]:
image_train_input['Probabilities'] = image_train_input['Probabilities'].str.replace('[', '')
image_train_input['Probabilities'] = image_train_input['Probabilities'].str.replace(']', '')

image_val_input['Probabilities'] = image_val_input['Probabilities'].str.replace('[', '')
image_val_input['Probabilities'] = image_val_input['Probabilities'].str.replace(']', '')

In [6]:
image_train_prob = []
for row in image_train_input['Probabilities']:
    probclass = row.split()
    probclass = list(map(float, probclass))
    image_train_prob.append(probclass)

image_val_prob = []
for row1 in image_val_input['Probabilities']:
    probclass1 = row1.split()
    probclass1 = list(map(float, probclass1))
    image_val_prob.append(probclass1)

In [7]:
image_train_prob = pd.DataFrame(image_train_prob)
image_val_prob = pd.DataFrame(image_val_prob)
#image_val_prob

In [8]:
image_train_prob.columns = [str(x) for x in range(275, 550)]
image_val_prob.columns = [str(x) for x in range(275, 550)]

The predicted probabilities per classifier are now concatenated, to create both datasets.

In [10]:
#The two output files of the base layer, with the text model and image model, are now added together, to serve as input.
total_train_input = pd.concat([text_train_input, image_train_prob], axis=1)
total_val_input = pd.concat([text_val_input, image_val_prob], axis=1)

total_train_input.columns = [str(x) for x in range(0, 550)]
total_val_input.columns = [str(x) for x in range(0, 550)]

#### Now the datasets can be applied to the models.
First, a GridSearch will be done for each combination of text classifier output and image classifier output (thus creating the 5 ensemble models) 

In [1]:
p_svc = {
    "estimator__C": [0.001, 0.01, 0.1, 1,10,100],
    "estimator__kernel": ["rbf", "linear", "poly", "sigmoid"],
    "estimator__gamma": [1, 0.1, 0.01, 0.001]
}


sk_SVC = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

clf3 = OneVsRestClassifier(SVC())

grid = GridSearchCV(clf3, p_svc, cv=sk_SVC, n_jobs = -1, verbose=3)
grid.fit(total_train_input, image_train_input['Label'])
grid.cv_results_
grid.best_params_

Having found the optimal parameters for each ensemble model (above), every meta-learner is trained and tested based on the combination of text and image classifier.

In [ ]:
#SVC with KNN input:
svcKNN = OneVsRestClassifier(SVC(C=1 , kernel='rbf', gamma=1, probability=True))
svcKNN.fit(total_train_input, image_train_input['Label'])
svcKNN_predicted = svcKNN.predict(total_val_input)
svcKNN_proba_val2 = svcKNN.predict_proba(total_val_input)
print("Acc:", accuracy_score(image_val_input['Label'], svcKNN_predicted))

In [11]:
#SVC with LR input:
svcLR = OneVsRestClassifier(SVC(C=1 , kernel='linear', gamma=1, probability=True))
svcLR.fit(total_train_input, image_train_input['Label'])
svcLR_predicted = svcLR.predict(total_val_input)
svcLR_proba_val2 = svcLR.predict_proba(total_val_input)
print("Acc:", accuracy_score(image_val_input['Label'], svcLR_predicted))

Acc: 0.9571865443425076


In [ ]:
#SVC with NB input:
svcNB = OneVsRestClassifier(SVC(C=100 , kernel='rbf', gamma=0.01, probability=True))
svcNB.fit(total_train_input, image_train_input['Label'])
svcNB_predicted = svcNB.predict(total_val_input)
svcNB_proba_val2 = svcNB.predict_proba(total_val_input)
print("Acc:", accuracy_score(image_val_input['Label'], svcNB_predicted))

In [ ]:
#SVC with SVM input:
svc = OneVsRestClassifier(SVC(C=1 , kernel='linear', gamma=1, probability=True))
svc.fit(total_train_input, image_train_input['Label'])
svc_predicted = svc.predict(total_val_input)
svc_proba_val2 = svc.predict_proba(total_val_input)
print("Acc:", accuracy_score(image_val_input['Label'], svc_predicted))

In [ ]:
#The output of the DT algorithm contains some NaN values, along with the predictions made.
#These will be set to zero, as these are supposed to be zero. 
train_input_dt = total_train_input.fillna(0)
val_input_dt = total_val_input.fillna(0)

In [ ]:
#SVC with DT input:
svcDT = OneVsRestClassifier(SVC(C=1 , kernel='linear', gamma=1, probability=True))
svcDT.fit(train_input_dt, image_train_input['Label'])
svcDT_predicted = svcDT.predict(val_input_dt)
svcDT_proba_val2 = svcDT.predict_proba(val_input_dt)
print("Acc:", accuracy_score(image_val_input['Label'], svcDT_predicted))

#### Calculate Top-5 Accuracy scores

In [12]:
## CODE COPIED FROM "Create TFRecords ipynb", to map the labels according to dictionary values (needed to make top-5 acc)
excel_labels275 = pd.read_excel('labels_left.xlsx')
excel_labels275 = excel_labels275.drop_duplicates()
list_labels275 = list(excel_labels275['labels_left'])

class_names_sorted = [int(x) for x in list_labels275]
class_names_sorted.sort(key=float)
class_names_string = [str(item) for item in class_names_sorted]
class_names_to_ids = dict(zip(class_names_string, range(len(class_names_string))))

#The mapping has to be done other way around, in this case
ids_to_class_names = {v: k for k, v in class_names_to_ids.items()}

In [13]:
def top5_acc(probability_predictions, ids_to_class_names):
    
    df_proba = pd.DataFrame(probability_predictions)
    prediction_top_proba = []
    for index, row in df_proba.iterrows():
        df_proba = df_proba = df_proba.rename(columns=ids_to_class_names)
        top_predictions = df_proba.sort_values(by=index, axis=1, ascending = False).columns.values[0:5]
        prediction_top_proba.append(top_predictions)

    prediction_top_proba = [list(map(int, row)) for row in prediction_top_proba]
    predictions_top_df = pd.DataFrame({'Y_val2': image_val_input['Label'], 'Y_top_proba': prediction_top_proba })
    total = 0
    for index, row in predictions_top_df.iterrows():
        real = predictions_top_df['Y_val2'].loc[index]
        predicted_list = predictions_top_df['Y_top_proba'].loc[index]
        true_pred = 0
        for prediction in predicted_list[0:5]:
            if real == prediction:
                true_pred = 1
        total += true_pred
    
    top5_acc = total/len(predictions_top_df)
    print('Accuracy %.2f'%(top5_acc*100))

In [ ]:
print("kNN + Inception Top-5:")
print(top5_acc(svcKNN_proba_val2, ids_to_class_names))
print("LogReg + Inception Top-5:")
print(top5_acc(svcLR_proba_val2, ids_to_class_names))
print("NaïveBayes + Inception Top-5:")
print(top5_acc(svcNB_proba_val2, ids_to_class_names))
print("SVC + Inception Top-5:")
print(top5_acc(svc_proba_val2, ids_to_class_names))
print("DT + Inception Top-5:")
print(top5_acc(svcDT_proba_val2, ids_to_class_names))

Calculate F1 Micro & F1 Macro for each ensemble model, with additional check to see whether F1-micro == accuracy (due to multi-class setting, this should be the case)

In [17]:
from sklearn.metrics import f1_score

def metrics_calc(pred):
    print("Acc: ", accuracy_score(image_val_input['Label'], pred))
    print("F1-micro: ", f1_score(image_val_input['Label'], pred, average = 'micro'))
    print("F1-macro: ", f1_score(image_val_input['Label'], pred, average = 'macro'))

In [ ]:
print("kNN + Inception F1 Metrics:")
print(metrics_calc(svcKNN_predicted))
print("LogReg + Inception F1 Metrics:")
print(metrics_calc(svcLR_predicted))
print("NaïveBayes + Inception Top-5:")
print(metrics_calc(svcNB_predicted))
print("SVC + Inception Top-5:")
print(metrics_calc(svc_predicted))
print("DT + Inception Top-5:")
print(metrics_calc(svcDT_predicted))